In [1]:
from ChatGLM3_router import ChatGLM3
from langchain.chains.router import MultiPromptChain
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [2]:
# 主要创建两个知识域的模板，一个营销文案作者，一个客服

Copywriter_template = """You are a master of persuasion and a pro at captivating audiences. \
You have the ability to craft captivating messages that move people to action. \
You know that the key to effective communication is understanding your audience and speaking directly to their needs and desires.\
You have rich writing experience and can write copywriting, advertisements, and soft articles with promotion and marketing intentions.
Here is a question:
{input}"""

RedWriter_template = """You are a Little Red Book blogger who is passionate about sharing your lifestyle and product reviews. \
You have a keen eye for trends and love to recommend new and exciting products to your followers. \
Whether it's the latest skincare must-have, a trendy new lip color, or your dream car, you can't wait to gush about your favorites and help others make informed purchase decisions. \
You actively engage with your followers, responding to comments and questions and building a loyal community around your shared interests.

Here is a question:
{input}"""

customerService_template = """You are a customer service representative. You are skilled at assisting customers and addressing their concerns. \
You take a problem-solving approach to help resolve issues and provide solutions. You should answer product, brand questions and purchase questions，and introduce product features.

Here is a question:
{input}"""


In [3]:

    
prompt_infos = [
    {
        "name": "Copywriter",
        "description": "Good for writing marketing copy , promotion copy and soft articles",
        "prompt_template": Copywriter_template,
    },
    {
        "name": "RedWriter",
        "description": "Good for writing something to recommend skin care products, cosmetics etc.",
        "prompt_template": RedWriter_template,
    },
    {
        "name": "customerService",
        "description": "Good for answering product, brand, purchase and product features questions",
        "prompt_template": customerService_template,
    },
]
model_path = "/data/tds/ChatGLM3/chatglm3-6b"
llm = ChatGLM3()
llm.load_model(model_name_or_path=model_path)

# 下面这段主要通过遍历得到destination_chains
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
# 得到一个起始chain
default_chain = ConversationChain(llm=llm, output_key="text")


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [4]:
from langchain.chains.router.embedding_router import EmbeddingRouterChain
from langchain.embeddings import CohereEmbeddings
from langchain.vectorstores import Chroma
import os
os.environ["COHERE_API_KEY"]="rAcrjO5IQn8Vrrg3NZQjzpv4beCMcp7vz2ZPA9Ei"
names_and_descriptions = [
    ("Copywriter", ["for questions about writing marketing copy , promotion copy and soft articles"]),
    ("RedWriter", ["for questions about recommend skin care products, cosmetics etc."]),
    ("customerService", ["for questions about product , brand, purchase and product features"]),
]

router_chain = EmbeddingRouterChain.from_names_and_descriptions(
    names_and_descriptions, Chroma, CohereEmbeddings(), routing_keys=["input"]
)


In [5]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [6]:
import pandas as pd
df = pd.read_excel("/home/kit/kit/baichuan/NewPrompt.xlsx",header=None)
df

,0,1
0,你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：\n\n产品名称： X品...,NaN
1,你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：\n\n产品名称： X品...,NaN
2,你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：\n\n产品名称： X品...,NaN
3,你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：\n\n产品名称： X品...,NaN
4,你是一名广告文案创作者，请根据以下要求创作宣传语：\n\n产品名称： LOHO新款眼睛\n\...,你是 品牌公关经理 ，请撰写一篇 微博 内容，宣布新产品的上市。\n\n开头： 提到上一个热...
5,你是一名广告文案创作者，请根据以下要求创作宣传语：\n\n产品名称： DW新款手表 \n\n...,你是 品牌公关经理 ，请撰写一篇 微博 内容，宣布新产品的上市。\n\n开头： 提到上一个热...
6,你是一名广告文案创作者，请根据以下要求创作宣传语：\n\n产品名称： 潘多拉新款项链\n\n...,你是 品牌公关经理 ，请撰写一篇 微博 内容，宣布新产品的上市。\n\n开头： 提到上一个热...
7,你是一名广告文案创作者，请根据以下要求创作宣传语：\n\n产品名称： 施华洛世奇新款耳环\n...,你是 品牌公关经理 ，请撰写一篇 微博 内容，宣布新产品的上市。\n\n开头： 提到上一个热...
8,你是一位优秀的短视频博主，请为新一期视频撰写脚本\n\n你的目标粉丝群体是： 想提升自己的学...,NaN
9,你是一位优秀的短视频博主，请为新一期视频撰写脚本\n\n你的目标粉丝群体是： 想提升自己的学...,NaN


In [7]:
for i in range(int(len(df))):
    print(
        chain.run(
            df.iloc[i,0]
        )
)



> Entering new MultiPromptChain chain...
customerService: {'input': '你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：\n\n产品名称： X品牌洗面奶 \n\n目标用户： 20-35岁的都市女性 \n\n内容要求： 自然融入产品参数和功能解读，结合用户故事或产品体验 \n\n创作要点： 隐蔽推广，文章主题与产品关联不明显 '}You are a customer service representative. You are skilled at assisting customers and addressing their concerns. You take a problem-solving approach to help resolve issues and provide solutions. You should answer product, brand questions and purchase questions，and introduce product features.

Here is a question:
你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：

产品名称： X品牌洗面奶 

目标用户： 20-35岁的都市女性 

内容要求： 自然融入产品参数和功能解读，结合用户故事或产品体验 

创作要点： 隐蔽推广，文章主题与产品关联不明显 

> Finished chain.



> Entering new MultiPromptChain chain...
customerService: {'input': '你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：\n\n产品名称： X品牌沐浴露 \n\n目标用户： 20-35岁的都市女性 \n\n内容要求： 自然融入产品参数和功能解读，结合用户故事或产品体验 \n\n创作要点： 隐蔽推广，文章主题与产品关联不明显'}You are a customer service representative. You are skilled at assisting customers and addressing their concerns. You 

KeyboardInterrupt: 

In [ ]:
print(
    chain.run(
        "你是一名汽车专家，需要为你的新车评测视频制作一个标题：\n\n视频主题： 哪吒S详细评测 \n\n视频内容： 深入分析哪吒S的性能、驾驶体验和能效 \n\n目标观众： 汽车爱好者和潜在购车者 "
    )
)

In [ ]:
print(
    chain.run(
        "水的化学式是什么"
    )
)